#  Sentiment Analysis with TensorFlow

Sentiment analysis is a very common text analytics task that involves determining whether a text sample is positive or negative about its subject.  There are several different algorithms for performing this task, including statistical algorithms and deep learning algorithms.  With respect to deep learning, a Convolutional Neural Net (CNN) is sometimes used for this purpose.  In this notebook we'll use a CNN built with TensorFlow to perform sentiment analysis in Amazon SageMaker on the IMDB dataset, which consists of movie reviews labeled as having positive or negative sentiment. Three aspects of Amazon SageMaker will be demonstrated:

- How to use Script Mode with a prebuilt TensorFlow container, along with a training script similar to one you would use outside SageMaker. 
- Batch Transform for offline, asynchronous predictions on large batches of data.
- How to use Automatic Model Tuning to optimize the performance of your ML model

#  Prepare Dataset

We'll begin by loading the reviews dataset, and padding the reviews so all reviews have the same length.  Each review is represented as an array of numbers, where each number represents an indexed word.  Training data must be saved as files, so we'll also save the transformed data to files.

In [1]:
import os
import boto3
import sagemaker
import numpy as np
from tensorflow.keras.preprocessing import sequence
from tensorflow.python.keras.datasets import imdb

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role() # we are using the notebook instance role for training in this example

In [2]:
max_features = 20000
maxlen = 400

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

17465344/17464789 [==============================] - 1s 0us/step
25000 train sequences
25000 test sequences
x_train shape: (25000, 400)
x_test shape: (25000, 400)


In [3]:
data_dir = os.path.join(os.getcwd(), 'data')
os.makedirs(data_dir, exist_ok=True)

train_dir = os.path.join(os.getcwd(), 'data/train')
os.makedirs(train_dir, exist_ok=True)

test_dir = os.path.join(os.getcwd(), 'data/test')
os.makedirs(test_dir, exist_ok=True)

csv_test_dir = os.path.join(os.getcwd(), 'data/csv-test')
os.makedirs(csv_test_dir, exist_ok=True)

In [4]:
import pandas as pd 
pd.DataFrame(x_train).to_csv(os.path.join(train_dir, 'x_train.csv'), header=None, index=False)
pd.DataFrame(y_train).to_csv(os.path.join(train_dir, 'y_train.csv'), header=None, index=False)
pd.DataFrame(x_test).to_csv(os.path.join(test_dir, 'x_test.csv'), header=None, index=False)
pd.DataFrame(y_test).to_csv(os.path.join(test_dir, 'y_test.csv'), header=None, index=False)
np.savetxt(os.path.join(csv_test_dir, 'csv-test.csv'), np.array(x_test[:100], dtype=np.int32), fmt='%d', delimiter=",")

#  Hosted Training

SageMaker's hosted training, which uses compute resources separate from your notebook instance.  Hosted training spins up one or more instances (cluster) for training, and then tears the cluster down when training is complete. In general, hosted training is preferred for doing actual training, especially for large-scale, distributed training. Before starting hosted training, the data must be present in storage that can be accessed by SageMaker. The storage options are:  Amazon S3 (object storage service), Amazon EFS (elastic NFS file system service), and Amazon FSx for Lustre (high-performance file system service). For this example, we'll upload the data to S3.  

In [5]:
bucket = 'sagemaker-eu-west-1-520691797693'
s3_prefix = 'tf-keras-sentiment'

traindata_s3_prefix = f'{s3_prefix}/data/train'
testdata_s3_prefix = f'{s3_prefix}/data/test'

train_s3 = sagemaker_session.upload_data(path='./data/train/', bucket=bucket, key_prefix=traindata_s3_prefix)
test_s3 = sagemaker_session.upload_data(path='./data/test/', bucket=bucket, key_prefix=testdata_s3_prefix)

inputs = {'train':train_s3, 'test': test_s3}
print(inputs)

{'train': 's3://sagemaker-eu-west-1-520691797693/tf-keras-sentiment/data/train', 'test': 's3://sagemaker-eu-west-1-520691797693/tf-keras-sentiment/data/test'}


## Automatic Model Tuning

In [7]:
from sagemaker.tensorflow import TensorFlow
from sagemaker.tuner import IntegerParameter, HyperparameterTuner

train_instance_type = 'ml.p3.2xlarge'
model_dir = '/opt/ml/model'

estimator = TensorFlow(entry_point='train.py',
                       source_dir='./training_scripts/',
                       model_dir=model_dir,
                       train_instance_type=train_instance_type,
                       train_instance_count=1,
                       role=role,
                       base_job_name='tf-keras-sentiment',
                       framework_version='1.13',
                       py_version='py3')

hyperparameter_ranges = {
        'epochs': IntegerParameter(10, 50),
        'batch_size': IntegerParameter(64, 256),
    }

metric_definitions = [{'Name': 'val_acc',
                       'Regex': ' val_acc: ([0-9\\.]+)'}]

objective_metric_name = 'val_acc'
objective_type = 'Maximize'

In [9]:
tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=10,
                            max_parallel_jobs=2,
                            objective_type=objective_type)

tuner.fit(inputs)